# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kirchjna\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kirchjna\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [2]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [3]:
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier

In [4]:
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.base import BaseEstimator, TransformerMixin

In [5]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///disaster.db')
    df = pd.read_sql_table('disaster_data', engine)

    X = df["message"]
    Y = df.iloc[:,4:]
    return X, y

engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql_table('disaster_data', engine)

X = df["message"]
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(tok)
        
    return clean_tokens

In [7]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [8]:
tokenize(X[0])

['Weather',
 'update',
 '-',
 'a',
 'cold',
 'front',
 'from',
 'Cuba',
 'that',
 'could',
 'pass',
 'over',
 'Haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
vect = CountVectorizer(tokenizer=tokenize)

In [10]:
#def model_pipeline():
#     pipeline = Pipeline([('vect', CountVectorizer(tokenizer = tokenize)),
#                      ('tfidf', TfidfTransformer()),
#                      ('clf', MultiOutputClassifier(RandomForestClassifier()))                
#                       ])
#     return pipeline
pipeline = Pipeline([('vect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier()))                
                      ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
#X, y = make_multilabel_classification(n_classes=36, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.33,
                                                    random_state=42)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000016B3E9DA318>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [13]:
y_pred = pipeline.predict(X_test)

In [14]:
y_test.shape

(8652, 36)

In [15]:
#classification_report(y_test.iloc[:,:], y_pred[:,:], labels = y_test.columns)

In [16]:
classification_report(y_test.iloc[:,1], y_pred[:,1])

'              precision    recall  f1-score   support\n\n           0       0.89      0.99      0.94      7180\n           1       0.88      0.43      0.58      1472\n\n    accuracy                           0.89      8652\n   macro avg       0.89      0.71      0.76      8652\nweighted avg       0.89      0.89      0.88      8652\n'

In [31]:
def display_results(cv, y_test, y_pred):
    for col in range(0,y_train.shape[1]):
        #print(y_train.iloc[:5,col], y_test[:5,col])
        print(classification_report(y_test.iloc[:,col], y_pred[:,col]))
    print("\nBest Parameters:", cv.best_params_)

### 6. Improve your model
Use grid search to find better parameters. 

In [32]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000016B3E9DA318>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000016B3E9DA318>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [33]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier()))                
                      ])

In [34]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 50, 100, 200],
        'clf__estimator__min_samples_leaf': [1,2, 3, 4]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [35]:
def build_model():
    pipeline = Pipeline([('vect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier()))                
                      ])

    parameters = {
            #'vect__ngram_range': ((1, 1), (1, 2)),
            'vect__max_df': (0.5, 0.75, 1.0),
            #'vect__max_features': (None, 5000, 10000),
            'tfidf__use_idf': (True, False),
            'clf__estimator__n_estimators': [10, 100],
            #'clf__estimator__min_samples_leaf': [2, 3, 4]
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [36]:
def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model = build_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    display_results(model, y_test, y_pred)
    


In [37]:
main()

              precision    recall  f1-score   support

           0       0.75      0.28      0.41      1548
           1       0.81      0.97      0.88      4950
           2       0.60      0.27      0.37        56

    accuracy                           0.80      6554
   macro avg       0.72      0.51      0.55      6554
weighted avg       0.79      0.80      0.77      6554

              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5476
           1       0.89      0.44      0.59      1078

    accuracy                           0.90      6554
   macro avg       0.89      0.72      0.77      6554
weighted avg       0.90      0.90      0.88      6554

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0

c:\users\kirchjna\anaconda3\envs\data_pipeline\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\kirchjna\anaconda3\envs\data_pipeline\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\kirchjna\anaconda3\envs\data_pipeline\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6220
           1       0.74      0.06      0.11       334

    accuracy                           0.95      6554
   macro avg       0.85      0.53      0.54      6554
weighted avg       0.94      0.95      0.93      6554

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      6359
           1       0.88      0.04      0.07       195

    accuracy                           0.97      6554
   macro avg       0.92      0.52      0.53      6554
weighted avg       0.97      0.97      0.96      6554

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6430
           1       0.00      0.00      0.00       124

    accuracy                           0.98      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.96      0.98      0.97      6554

              preci

c:\users\kirchjna\anaconda3\envs\data_pipeline\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\kirchjna\anaconda3\envs\data_pipeline\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\kirchjna\anaconda3\envs\data_pipeline\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6255
           1       0.00      0.00      0.00       299

    accuracy                           0.95      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.91      0.95      0.93      6554

              precision    recall  f1-score   support

           0       0.87      0.96      0.91      4738
           1       0.86      0.63      0.73      1816

    accuracy                           0.87      6554
   macro avg       0.87      0.80      0.82      6554
weighted avg       0.87      0.87      0.86      6554

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6015
           1       0.92      0.39      0.54       539

    accuracy                           0.95      6554
   macro avg       0.93      0.69      0.76      6554
weighted avg       0.95      0.95      0.94      6554

              preci

In [ ]:
Best Parameters: {'clf__estimator__n_estimators': 100, 'tfidf__use_idf': True, 'vect__max_df': 0.5}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.